In [ ]:
import easyocr
import cv2
from ultralytics import YOLO
import os
import csv

def pipeline(input_folder, model_path='model_yolo.pt', model_ocr_path='mode_easyocr.pth'):
    
    model_ocr = easyocr.Reader(['ru', 'en'], gpu=False, recognizer=model_ocr_path)

    model = YOLO(model_path)

    # Проходим по всем изображениям в папке ввода
    with open('output.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["image_file", "label", "label_text"])
        for image_name in os.listdir(input_folder):
            if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                continue
            # Определяем путь к изображению
            image_path = os.path.join(input_folder, image_name)
            
            image = cv2.imread(image_path)

            # Преобразуем изображение в оттенки серого
            gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Применяем медианный фильтр
            median_image = cv2.medianBlur(gray_img, 5)

            image_preprocess = cv2.Laplacian(median_image, cv2.THRESH_BINARY, ksize=5, scale=0.55)

            image_preprocess = cv2.cvtColor(image_preprocess, cv2.COLOR_GRAY2BGR)  # Преобразуем в трёхканальное изображение

            my_conf = 0.8
            results = model.predict(source=image_preprocess, save=False, save_txt=False, conf=my_conf)  # conf=0.5 - минимальная вероятность обнаружения
            while (len(results[0].boxes) == 0):
                my_conf -= 0.05
                results = model.predict(source=image_preprocess, save=False, save_txt=False, conf=my_conf)  # conf=0.5 - минимальная вероятность обнаружения

            height, width, _ = image_preprocess.shape
            bbox = ''
            result_final = []
            result = results[0].boxes[0]
            
            x1, y1, x2, y2 = map(int, result.xyxy[0])  # Координаты рамки (xmin, ymin, xmax, ymax)
            class_id = int(result.cls[0])  # ID класса
            label = model.names[class_id]  # Получение метки класса

            # Вычисление центра и размеров рамки
            x_center = (x1 + x2) / (2 * width)
            y_center = (y1 + y2) / (2 * height)
            bbox_width = (x2 - x1) / width
            bbox_height = (y2 - y1) / height
            bbox += f'{0} {x_center} {y_center} {bbox_width} {bbox_height}\n'
            # Форматированный вывод координат
            print(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")

            # Обрезка изображения по найденной рамке
            cropped_image = image[y1:y2, x1:x2]  # Обрезаем изображение

            # Сохранение обрезанного изображения
            # output_path = os.path.join(output_folder, f"cropped_{str(i) + image_name}")  # Используем i + 1 для нумерации
            
            result = model_ocr.readtext(cropped_image, rotation_info=[90, 180 ,270], 
                                    detail=0, batch_size=32, text_threshold=0.5, low_text=0.2, 
                                    allowlist='139-08ТС.456Пр27АМГ/чотНЕОВLPЗИКРвБAMлмаксионOZI[]бКУЭmaxхinФ')
            if len(result) > 1:
                result = [result[0], result[-1]]
            result_final += result
                # first = result[0]
                # last = result[-1]

                
            writer.writerow([image_name, bbox, f'{"".join(result_final)}'])

pipeline('dataset/val/images')


Using CPU. Note: This module is much faster with a GPU.



0: 640x480 1 0, 129.5ms
Speed: 1.4ms preprocess, 129.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 480)
0 0.463542 0.425781 0.360417 0.198437

0: 640x480 1 0, 114.1ms
Speed: 0.9ms preprocess, 114.1ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)
0 0.400521 0.487891 0.726042 0.355469

0: 480x640 2 0s, 120.1ms
Speed: 0.8ms preprocess, 120.1ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
0 0.067187 0.290104 0.134375 0.373958

0: 640x480 1 0, 115.8ms
Speed: 1.0ms preprocess, 115.8ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)
0 0.491667 0.504297 0.464583 0.344531

0: 640x480 2 0s, 117.8ms
Speed: 1.2ms preprocess, 117.8ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)
0 0.679739 0.556373 0.158170 0.202941

0: 288x640 2 0s, 74.3ms
Speed: 0.7ms preprocess, 74.3ms inference, 0.2ms postprocess per image at shape (1, 3, 288, 640)
0 0.732031 0.472902 0.375000 0.620629

0: 640x480 1 0, 118.8ms
Speed: 0